In [ ]:
# IMPORTS

import sys
import os
import time
from datetime import datetime
from datetime import timedelta
from datetime import date
import json
import numpy as np
import pandas as pd
import bob
import openpyxl
from pymongo import ASCENDING
from pymongo import DESCENDING
import plotly.express as px

In [1]:
# VARS
#-> Repository
INPUT_FOLDER = os.path.join('..', 'data_input')
OUTPUT_FOLDER = os.path.join('..', 'data_output')
PARAM_FOLDER = os.path.join('..', '..', '_parameters')
#-> MONGO
USE_MONGO = False
USE_HEALTHCHECKS = True
CREDENTIALS = os.path.join(PARAM_FOLDER, 'mongodb.json')
DB_SRC = 'covid-19-dev'
DB_APP = 'app-wsr-dev'
#-> Other
LOGO_CS = os.path.join(INPUT_FOLDER, "cashstory_all_white.png")
LOGO_CS = r"https://bob.cashstory.com/assets/images/cashstory_all_white.png"

In [2]:
#--- STEP0 --- : Init

#-> Date
NOW = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
TODAY = date.today()
YESTERDAY = (TODAY - timedelta(days=1))
B_YESTERDAY = (TODAY - timedelta(days=2))

DB_CON = None
MONGO_DB = None
if USE_MONGO:
    with open(CREDENTIALS) as f_cred:
        DB_CON = json.load(f_cred)
    MONGO_HOST = DB_CON["host"]
    MONGO_PORT = DB_CON["port"]
    MONGO_USERNAME = DB_CON["username"]
    MONGO_PASSWORD = DB_CON["password"]
    #-> Bob connect
    MONGODB = bob.mongo.connect(MONGO_HOST, MONGO_PORT,
                                MONGO_USERNAME, MONGO_PASSWORD)

REF_FRANCE = pd.read_excel(os.path.join(INPUT_FOLDER, 'REF_WSR.xlsx'),
                           sheet_name='REF_FRANCE')

In [ ]:
#-- Map chart
#US - 500K
JSON_FILE = os.path.join(INPUT_FOLDER, 'usa_states.json')
with open(JSON_FILE) as f:
    STATES = json.load(f)

#US - 20m
JSON_FILE = os.path.join(INPUT_FOLDER, 'usa_states_20m.json')
with open(JSON_FILE) as f:
    STATES20M = json.load(f)

#US - 5m
JSON_FILE = os.path.join(INPUT_FOLDER, 'usa_states_5m.json')
with open(JSON_FILE) as f:
    STATES5M = json.load(f)

#France - Region
JSON_FILE = os.path.join(INPUT_FOLDER, 'france-region.json')
with open(JSON_FILE) as f:
    FRANCE_REG = json.load(f)

#France - Region
JSON_FILE = os.path.join(INPUT_FOLDER, 'france-departement.json')
with open(JSON_FILE) as f:
    FRANCE_DEP = json.load(f)

#World map
JSON_FILE = os.path.join(INPUT_FOLDER, 'world-med.json')
with open(JSON_FILE) as f:
    WORLDMAP = json.load(f)

def analysis(dataf, variable):
    """
    doc string
    """
    val_min = dataf[variable].min()
    val_max = dataf[variable].max()
    avr = round(dataf[variable].mean(), 0)
    med = round(np.median(dataf[variable]), 0)
    avrmax = round((avr / val_max), 2)
    medmax = round((med / val_max), 2)
    to_print = (f'{variable} : Min: {val_min}, Max: {val_max}, Average: {avr}, '
                f'AvrMax: {avrmax}, Med: {med}, MedMax: {medmax}')
    return print(to_print)

#Fonction to add CSS
def update_chart_css(filename, css_filename):
    """
    Function to add CSS
    """
    a_map = None
    css = None
    with open(filename) as file:
        a_map = file.read()

    with open(css_filename) as file:
        css = file.read()
    if a_map.find('id="cs_css"') != -1:
        print("to do")
    else:
        result = a_map.replace("<body>", f'<body><style id="cs_css">{css}</style>')
        with open(filename, "w") as file:
            file.write(result)
            file.close()

In [ ]:
#-- Datasource : Check if url exists and return df
# default seperator = ','
def check_url(url, separator=','):
    """
    Datasource : Check if url exists and return df
    default seperator = ','
    """
    dataf = pd.DataFrame()
    try:
        dataf = pd.read_csv(url, sep=separator)
    except Exception as exc:
        print(f'Error connecting to {url}.')
        print(exc.__doc__)
        print(str(exc))
    return dataf

In [ ]:
#-- Datasource local : open csv in local if path exists or open DB MONGO
def get_datasource(src_name):
    """
    Datasource local : open csv in local if path exists or open DB MONGO
    """
    src_path = os.path.join(OUTPUT_FOLDER, src_name + '.csv')
    if os.path.exists(src_path):
        dataf = pd.read_csv(src_path, sep=';', low_memory=False)
        if dataf and 'Unnamed: 0' in dataf.columns:
            dataf = dataf.drop('Unnamed: 0', axis=1)
    else:
        dataf = bob.mongo.read_df(src_name, DB_SRC)
        if dataf:
            dataf = dataf.drop('_id', axis=1)
        else:
            print(f'Warning ! Source {src_name} does not exist in csv or in DB')
    return dataf

In [ ]:
#-- Trend : Create date scenario
#col scenario default : 'DATE_SCENARIO'
def get_lastdays(dataf, limit, title, col='DATE_SCENARIO'):
    """
    Trend : Create date scenario
    col scenario default : 'DATE_SCENARIO'
    """
    df_b = pd.DataFrame()
    if 'DATE_ORDER' in dataf.columns:
        all_dates = (dataf['DATE_ORDER'].sort_values(ascending=False).
                     drop_duplicates().values.tolist()[:limit])
    for each in all_dates:
        tmp_df = dataf[dataf['DATE_ORDER'] == each]
        df_b = df_b.append(tmp_df)
        df_b[col] = title
    return df_b